In [34]:
# Run this code block for CSS styling.
from IPython.core.display import HTML

HTML(open("./Styles/custom.css", "r").read())

<p class='notebook_header'><b>CS 309 - Robot Learning</b></p>
<p class='notebook_header'>Robosuite Introduction</p>
<p>Due to the pandemic, we may not be able to access the robot. Instead, we can simulate the robot via Robosuite, a simulation framework for robot learning that's partially developed at UT.</p>
<p>This assignment aims to get you familiar with this framework, so that you will have an eaiser time using it later in this course.</p>
<hr class='separate' />

<p class="section_header"><b>Part I: Installation</b></p>

<p>Please follow the directions here to install robosuite: <a href="https://robosuite.ai/docs/installation.html">https://robosuite.ai/docs/installation.html</a>.</p>
Notes: 
<ol>
    <li>When getting a Mujoco license, make sure you choose the personal student license, which is free for a year. It might take a day or so to approve your account, so get started early! </li>
    <li>You can use a conda environment to organize your packages. Do NOT use python 3.9 - it did not work for me.</li>
    <li>If you are on Mac and you run into this error, '<em>RuntimeError: Could not find GCC executable</em>', try doing <code>brew install gcc@8</code>. For more information, visit <a href="https://github.com/openai/mujoco-py/issues/254">this site</a>.</li>
    <li>If you are on a Mac and your system blocks mujoco from running ('library load disallowed by system policy'), use <code>xattr -d com.apple.quarantine &lt;name&gt;.</li>
    <li>If you choose to install from pip, the command with which to test your installation <code>python -m robosuite.demos.demo_random_action</code> did not work for me. However, the installation is still usable. You can make sure by running the script provided in <a href="https://robosuite.ai/docs/quickstart.html">quick start</a>. </li>
</ol>

<p class="section_header"><b>Part II: Introduction</b></p>
To get a brief overview of the framework, please read the following pages in the documentation. These concepts will be important for you to use this framework.
<ol>
    <li><a href="https://robosuite.ai/docs/overview.html">Basic introduction</a></li>
    <li><a href="https://robosuite.ai/docs/modules/overview.html">An overview of the robosuite architecture</a></li>
    <li><a href="https://robosuite.ai/docs/modules/robots.html">An overview of robots</a></li>
    <li><a href="https://robosuite.ai/docs/modules/objects.html">An overview of objects</a></li>
    <li><a href="https://robosuite.ai/docs/modules/environments.html">An overview of environments</a></li>
    <li>You are also encouraged to read the rest of the "Modules" sections of the documentation, but we won't use them in this assignment.</li>
</ol>

<p class="section_header"><b>Part III: Getting Started</b></p>

Now, let's get our hands dirty.

First, let's try teleoperating the robot with a keyboard.

Download <a href="https://github.com/ARISE-Initiative/robosuite/blob/master/robosuite/demos/demo_device_control.py">this script</a> and run it. I recommend running it in a separate python file since it's easier. If you want to run it in a cell in this notebook, you have to make sure that your jupyter notebook is using the python kernel that you installed robosuite on. Also, if the code throws an error about unrecognized arguments in jupyter, you can add a dummy input variable <code>parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")</code> to the list of parser arguments (according to <a href="https://stackoverflow.com/questions/48796169/how-to-fix-ipykernel-launcher-py-error-unrecognized-arguments-in-jupyter">this post</a>).

It's possible that it will throw an error about SpaceMouse. The workaround I found was to delete the reference to SpaceMouse in the utils/input_utils.py file. Namely, change line 246 and 247 to <code>drotation = drotation * 1.5 dpos = dpos * 75</code>.
If you have installed robosuite with pip, you can find the utils folder in the site-packages folder of your python installation. You can find the site-packages folder through <code>python -m site</code>. For example, mine is <code>miniconda3/envs/robosuite/lib/python3.8/site-packages/</code>.

If the code successfully runs, then you should be able to control the robot arm with your keyboard. 

<hr/>



Now, let's try creating your own environment.

Follow <a href="https://robosuite.ai/docs/quickstart.html#building-your-own-environments">this guide</a> to create your own environment. Make sure you understand what each line of the code does. 

As of right now, the step 4 isn't compliant with the current API. 
You should use get_obj() to get the XML tree instead of get_collision(). Also, objects by default have a free joint, so you shouldn't append a new joint. Therefore, the code should be <code>
    sphere = BallObject(
    name="sphere",
    size=[0.04],
    rgba=[0, 0.5, 0.5, 1]).get_obj()
    sphere.set('pos', '1.0 0 1.0')
    world.worldbody.append(sphere)
</code>I have reported this issue to the robosuite team, but they might not have addressed it yet.

If your code runs, you should be able to see your custom environment rendered in an interactive window. 

Be prepared to show your custom environment during office hour.
<hr/>

Below are some python cells to help you explore the robosuite API. Please run each cell and make sure that you understand them. Feel free to experiment with the code and insert print statements to examine the data structures.

Again, make sure that your python kernel for Jupyter Notebook has robosuite installed.

Credits to <a href="https://github.com/ARISE-Initiative/robosuite-notebooks/blob/master/robosuite_101.ipynb">Yuke Zhu</a> for the code.

In [ ]:
import numpy as np
import robosuite as suite

from PIL import Image
from IPython.display import display

In [ ]:
suite.ALL_ENVIRONMENTS

In [ ]:
suite.ALL_ROBOTS

In [ ]:
suite.ALL_GRIPPERS

In [ ]:
suite.ALL_CONTROLLERS

In [ ]:
env = suite.make(
    env_name="Lift",
    robots="Sawyer",
    has_renderer=False,
    ignore_done=True,
    use_camera_obs=True,
    camera_names=["frontview", "agentview"]
)

obs = env.reset()

In [ ]:
frontview = Image.fromarray(obs['frontview_image'][::-1])
display(frontview)

In [ ]:
agentview = Image.fromarray(obs['agentview_image'][::-1])
display(agentview)

In [ ]:
low, high = env.action_spec # get action limits
for i in range(100):
    action = np.random.uniform(low, high) # sample random action
    obs, reward, done, _ = env.step(action)
    
display(Image.fromarray(obs['frontview_image'][::-1]))

In [ ]:
print('number of bodies:', env.sim.model.nbody)
print('number of joints:', env.sim.model.njnt)
print('number of generalized coordinates:', env.sim.model.nq)
print('number of degrees of freedom:', env.sim.model.nv)
print('number of degrees of freedom:', env.sim.model.nu)
print('number of activation states:', env.sim.model.na)

In [ ]:
print(env.sim.model.body_names)

In [ ]:
body_id = 1

print(env.sim.model.body_names[body_id])
print(env.sim.data.body_xpos[body_id])
print(env.sim.data.body_xquat[body_id])

body_id = -1

print()
print(env.sim.model.body_names[body_id])
print('Frame origin:\n', env.sim.data.body_xpos[body_id])
print('\nRotation quaternion:\n', env.sim.data.body_xquat[body_id])
print('\nRotation matrix:\n', env.sim.data.body_xmat[body_id].reshape(3,3))

In [ ]:
# simple example of coordinate transformations
import robosuite.utils.transform_utils as T

T.quat2mat(T.convert_quat(np.array(env.sim.data.body_xquat[body_id]), to="xyzw"))

In [ ]:
env.sim.data.body_xmat[body_id].reshape(3, 3)

In [ ]:
env.sim.data.body_xpos[body_id]

In [ ]:
# get information of all bodies
for i in range(env.sim.model.nbody):
    name = env.sim.model.body_names[i]
    body_id = env.sim.model.body_name2id(name)
    body_xpos = env.sim.data.body_xpos[body_id]
    print(body_id, name, body_xpos)

<p class="section_header"><b>Part IV: Record Human Demonstration</b></p>

The primary topic of this research class is learning by demonstration. With robosuite, it's really easy to record human demonstrations.

In this final part of the assignment, you will learn to record and play back a human demonstration.

Using the collect human demonstration and play back demonstrations from hdf5.py scripts from <a href="https://github.com/ARISE-Initiative/robosuite/tree/master/robosuite/scripts">here</a>, record and play back an episode of the Stack environment.

Make sure to read the comments in source code to understand how to use them. If you have followed part III, you should not have any issues with the SpaceMouse error. 

Get ready to show the playback of your demontration during office hours.